## Text summarization (Sentence Ranking)

## Initial Phase
### Importing Libraries and Reading Data 

In [1]:
### importing the necessary libraries

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import networkx as nx
from nltk.tokenize import  sent_tokenize

In [2]:
# Baca data
df = pd.read_csv('Data/tennis_articles_v4.csv')
articles = df['article_text']

In [3]:
## working with re ( regular expression in python)

import re
s = 'he&&&s'
s = re.sub("[^a-zA-Z]"," ",s)

## Data Cleaning

In [4]:
# Bersihkan data
cleaned_sentences = []

for article in articles:
    article = article.lower()
    sentences = sent_tokenize(article)
    for sentence in sentences:
        clean_sentence = re.sub("[^a-zA-Z]", " ", sentence.lower())
        cleaned_sentences.append(clean_sentence)


## Building Senetnce Similarity Matrix


In [5]:
# Bangun Matriks Kemiripan Kalimat
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w for w in sent1]
    sent2 = [w for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix


## Sentence Ranking

In [6]:
sentence_similarity_martix = build_similarity_matrix(cleaned_sentences, '')

    # Step 3 - Rank sentences in similarity martix
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(cleaned_sentences)), reverse=True)  

## Summary Generation

In [9]:
# Sort dan ambil 1 kalimat teratas
ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(cleaned_sentences)), reverse=True)

# Outputkan summary dari 1 kalimat teratas
print("Summary:")
print(ranked_sentence[0][1])

Summary:
argentina and britain received wild cards to the new look event  and will compete along with the four      semi finalists and the    teams who win qualifying rounds next february 
